In [45]:
from pyspark.sql import SparkSession
from operator import add

spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.113:7077") \
        .appName("esraa_mohammed_a")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

spark_context.setLogLevel("INFO")


#A.1.1 :

In [2]:
elines = spark_context.textFile("hdfs://192.168.2.113:9000/europarl/europarl-v7.fi-en.en")
elines.count()


1924942

#A.1.2:

In [3]:
flines = spark_context.textFile("hdfs://192.168.2.113:9000/europarl/europarl-v7.fi-en.fi")
flines.count()

1924942

#A.1.3 :
total number of lines for both languages are : 1924942

#A.1.4: 3 partitions

In [12]:
print(elines.getNumPartitions())
print(flines.getNumPartitions())

3
3


#A.2.1

In [24]:
def preprocess(lines):
    lower = lines.map(lambda line : line.lower())\
                 .flatMap(lambda s: s.split(' '))\
                 .flatMap(lambda w: w.split('\n'))\
                 .map(lambda w: w.strip())\
                 .map(lambda w: (w,1))\
                 .reduceByKey(add)
    
    return lower

#A.2.2

In [25]:
epre = preprocess(elines)
print(epre.take(5))

[('of', 1724191), ('the', 3631865), ('session', 2430), ('declare', 1438), ('european', 278293)]


In [26]:
fpre = preprocess(flines)
print(fpre.take(5))

[('uudelleenavaaminen', 80), ('17.', 766), ('keskeytetyn', 231), ('avatuksi', 11), ('ja', 1249156)]


#A.2.3

In [16]:
print(flines.count())
print(elines.count())

1924942
1924942


#A.3.1

In [27]:
print(epre.takeOrdered(10, key=lambda x: -x[1]))
print(fpre.takeOrdered(10, key=lambda x: -x[1]))

[('the', 3631865), ('of', 1724191), ('to', 1600135), ('and', 1339070), ('in', 1127220), ('that', 830098), ('a', 803908), ('is', 785410), ('for', 553730), ('we', 548087)]
[('ja', 1249156), ('on', 1035956), ('että', 619655), ('euroopan', 257568), ('ei', 246268), ('myös', 178765), ('ovat', 161869), ('se', 152857), ('arvoisa', 149589), ('ole', 134745)]


#A.3.2 :

Most of the frequently used words in english are articles, pronounce, propositions and verb to be. same in French language

In [8]:
elines.map(lambda line: line.split(' ')).filter(lambda w: len(w) < 7).take(10)

[['Resumption', 'of', 'the', 'session'],
 ['Why', 'are', 'there', 'no', 'fire', 'instructions?'],
 ['Why', 'are', 'no-smoking', 'areas', 'not', 'enforced?'],
 ['Agenda'],
 ['Relating', 'to', 'Wednesday:'],
 ['(Applause', 'from', 'the', 'PSE', 'Group)'],
 ['That', 'was', 'the', 'decision.'],
 ['There', 'is', 'no', 'such', 'document!'],
 ['We', 'have', 'agreed', 'to', 'this.'],
 ['(Applause', 'from', 'the', 'PPE-DE', 'Group)']]

In [9]:
flines.map(lambda line: line.split(' ')).filter(lambda w: len(w) < 7).take(10)

[['Istuntokauden', 'uudelleenavaaminen'],
 ['(Parlamentti', 'vietti', 'seisaallaan', 'minuutin', 'hiljaisuuden.)'],
 ['Arvoisa', 'puhemies,', 'käytän', 'työjärjestyspuheenvuoron.'],
 ['Arvoisa', 'puhemies,', 'käytän', 'työjärjestyspuheenvuoron.'],
 ['Se', 'on', 'tapaus', 'Aleksandr', 'Nikitin.'],
 ['Toivon,', 'että', 'sitä', 'käsitellään', 'myönteisessä', 'hengessä.'],
 ['Hyvä', 'kollega,', 'tarkistamme', 'asian.'],
 ['Vaarana', 'todellakin', 'on', 'tuleva', 'sotilasvallankaappaus.'],
 ['Me', 'emme', 'tiedä,', 'mitä', 'tapahtuu.'],
 ['Käsittelyjärjestys']]

#A.4

In [43]:
en_1 = elines.zipWithIndex()
fr_1 = flines.zipWithIndex()

print(en_1.take(2))
print(fr_1.take(2))

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: java.lang.IllegalStateException: SparkContext has been shutdown
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2091)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2120)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2139)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2164)
	at org.apache.spark.rdd.RDD.$anonfun$collect$1(RDD.scala:1004)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:388)
	at org.apache.spark.rdd.RDD.collect(RDD.scala:1003)
	at org.apache.spark.api.python.PythonRDD$.collectAndServe(PythonRDD.scala:168)
	at org.apache.spark.api.python.PythonRDD.collectAndServe(PythonRDD.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.re

In [32]:
en_2 = en_1.map(lambda x: (x[1], x[0]))
fr_2 = fr_1.map(lambda x: (x[1], x[0]))

print(en_2.take(2))
print(fr_2.take(2))

[(0, 'Resumption of the session'), (1, 'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.')]
[(0, 'Istuntokauden uudelleenavaaminen'), (1, 'Julistan perjantaina joulukuun 17. päivänä keskeytetyn Euroopan parlamentin istunnon avatuksi ja esitän vielä kerran vilpittömän toiveeni siitä, että teillä olisi ollut oikein mukava joululoma.')]


In [41]:
en_3 = en_2.join(fr_2)

print(en_2.join(fr_2).take(2))

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: java.lang.IllegalStateException: SparkContext has been shutdown
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2091)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2120)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2139)
	at org.apache.spark.api.python.PythonRDD$.runJob(PythonRDD.scala:154)
	at org.apache.spark.api.python.PythonRDD.runJob(PythonRDD.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.inv